In [41]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn
!pip3 install scipy

In [60]:
import pandas as pd

In [95]:
!ls

airline_tweets.csv  features.csv   model.pkl	requirements.txt  venv
cities.csv	    Kinaxis.ipynb  pipeline.py	run.sh
clean_data.csv	    main.py	   README.md	scores.csv


In [96]:
df_airline = pd.read_csv("airline_tweets.csv", engine='python')
df_cities = pd.read_csv("cities.csv")

In [97]:
df_airline.head(3)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)


In [98]:
df_airline.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'airline_sentiment',
       'airline_sentiment:confidence', 'negativereason',
       'negativereason:confidence', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [99]:
df_airline.drop(columns=['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at',
       'airline_sentiment:confidence', 'negativereason',
       'negativereason:confidence', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'], inplace=True)

## `CleanDataTask`

In [100]:
# Cleaning the airline dataframe

# Removing rows cantaining NAN
df_airline.dropna(inplace=True)
# df_airline.reset_index(drop=True, inplace= True)

# Removing rows containing '0.0, 0.0'
df_airline['tweet_coord'] = df_airline['tweet_coord'].astype(str)
df_airline.drop(df_airline[df_airline.tweet_coord == '[0.0, 0.0]'].index, inplace=True)
df_airline.reset_index(drop=True, inplace= True)


In [101]:
df_airline.describe()

,airline_sentiment,tweet_coord
count,855,855
unique,3,831
top,negative,"[40.64656067, -73.78334045]"
freq,573,6


In [102]:
df_airline.dtypes

airline_sentiment    object
tweet_coord          object
dtype: object

In [103]:


# df_airline = pd.concat([df_airline,pd.get_dummies(df_airline['nearest_city'], prefix='nearest_city',drop_first=True)],axis=1)

# # now drop the original 'country' column (you don't need it anymore)
# df_airline.drop(['nearest_city'],axis=1, inplace=True)

## `TrainingDataTask`

In [104]:
# Converting list as string to list
import ast
df_airline['tweet_coord'] = df_airline['tweet_coord'].apply(lambda x: ast.literal_eval(x))

In [105]:
df_cities.drop(columns=['geonameid', 'asciiname', 'alternatenames',
                        'feature class', 'feature code', 'country code', 'cc2',
                        'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code',
                        'population', 'elevation', 'dem', 'timezone', 'modification date'], inplace= True)

In [106]:
df_cities.columns

Index(['name', 'latitude', 'longitude'], dtype='object')

In [107]:
len(set(df_cities['name']))

22162

In [111]:
# Dropping the dublicate values for the cities
df_cities = df_cities.drop_duplicates(subset='name', keep="first")
df_cities.reset_index(drop=True, inplace= True)

In [112]:
# combining the latitude and longitude columns of the dataframe
df_cities['latlong'] = df_cities.apply(lambda x: list([x['latitude'],
                                        x['longitude']]),axis=1)

In [113]:
# Nearest city logic
from scipy.spatial import KDTree
import numpy as np
s1 = np.array(list(df_cities['latlong']))
s2 = np.array(list(df_airline['tweet_coord']))

kdtree = KDTree(s1)
neighbours = kdtree.query(s2)

nearest_city = []
for i in range(len(neighbours[1])):
    nearest_city.append(df_cities['name'][i])
    
df_airline['nearest_city'] = nearest_city

In [ ]:
len(set(neighbours[1]))

In [ ]:
len(neighbours[1])

In [ ]:
len(set(nearest_city))

In [ ]:
s1.shape

In [ ]:
df_airline.columns

In [ ]:
len(set(df_airline['nearest_city']))

In [57]:
df_airline.columns

Index(['airline_sentiment', 'tweet_coord'], dtype='object')

In [58]:
print(df_airline.groupby(['airline_sentiment']).size())
df_airline = df_airline.drop(df_airline[df_airline['airline_sentiment'] == 'negative'].sample(frac=.78).index)
df_airline = df_airline.drop(df_airline[df_airline['airline_sentiment'] == 'positive'].sample(frac=.15).index)
print(df_airline.groupby(['airline_sentiment']).size())
df_airline.reset_index(drop=True, inplace= True)

airline_sentiment
negative    573
neutral     132
positive    150
dtype: int64
airline_sentiment
negative    126
neutral     132
positive    128
dtype: int64


In [59]:
# To view the full screen
pd.options.display.max_rows = 10000 

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y = le.fit_transform(df_airline['airline_sentiment'])


# print(list(le.inverse_transform([2, 2, 1])))

In [ ]:
# One hot encoding 

X = pd.get_dummies(df_airline["nearest_city"],prefix='nearest_city',drop_first=True)

df = pd.concat([df_airline,pd.get_dummies(df_airline['nearest_city'], prefix='nearest_city',drop_first=True)],axis=1)


In [ ]:
df.columns

In [ ]:
df = df.drop(columns = ['airline_sentiment', 'tweet_coord', 'nearest_city'])

In [ ]:
# https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()

# Train the model using the training sets
model.fit(X,y)

#Predict Output
# predicted= model.predict([[0,2]]) # 0:Overcast, 2:Mild
# print "Predicted Value:", predicted

In [ ]:
model.predict(X)

In [ ]:
model.predict_proba(X)

In [ ]:
model.score(X,y)

In [ ]:
# example of training a final classification model
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
# generate 2d classification dataset
# X, y = make_blobs(n_samples=100,random_state=1)
# fit final model
model = LogisticRegression()
model.fit(X, y)

In [ ]:
model.predict_proba(X)

In [ ]:
print(list(le.inverse_transform(model.predict(X))))

In [ ]:
model.score(X,y)

In [ ]:
from sklearn import svm

regr = svm.SVR()
regr.fit(X, y)

In [ ]:
model.predict(X)

In [ ]:
model.predict_proba(X)

In [ ]:
y

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=1)
clf.fit(X, y)

In [ ]:
model.predict(X)

In [ ]:
model.predict_proba(X)

In [ ]:
X_test = pd.get_dummies(df_cities["name"],prefix='name',drop_first=True)

In [ ]:
model.predict(X_test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=1)
clf.fit(X_train, y_train)

In [ ]:
# Get the probability for each prediction
import numpy as np
index_array = np.argmax(model.predict_proba(X_test),  axis=-1)
prediction_probability = np.take_along_axis(model.predict_proba(X_test), np.expand_dims(index_array, axis=-1), axis=-1)

In [ ]:
# Prediction on the test set
prediction = list(le.inverse_transform(model.predict(X_test)))

In [ ]:
city = list(X_test.idxmax(axis=1))

In [ ]:
prediction_proba=[ i[0] for i in prediction_probability.tolist()]

In [ ]:
dict = {'city': city, 'prediction': prediction, 'prediction_proba': prediction_proba}
df_test = pd.DataFrame(dict)

In [ ]:
df_test = df_test[df_test['prediction'] == 'positive']

In [ ]:
df_test.to_csv('City_')

In [ ]:
# class TrainingDataTask(luigi.Task):
#     """ Extracts features/outcome variable in preparation for training a model.
#
#         Output file should have columns corresponding to the training data:
#         - y = airline_sentiment (coded as 0=negative, 1=neutral, 2=positive)
#         - X = a one-hot coded column for each city in "cities.csv"
#     """
#     tweet_file = luigi.Parameter()
#     cities_file = luigi.Parameter(default='cities.csv')
#     output_file = luigi.Parameter(default='features.csv')
#
#     # TODO...
#     def requires(self):
#         return CleanDataTask()
#
#     def run(self):
#         df_clean = pd.read_csv(CleanDataTask().output().path)
#         # Converting list as string to list
#
#         df_cities = pd.read_csv(self.cities_file)
#         # combining the latitude and longitude columns of the dataframe
#         df_cities['latlong'] = df_cities.apply(lambda x: list([x['latitude'], x['longitude']]), axis=1)
#
#         # Using the KDtree to locate the nearest city for the respective coordinate
#         kdtree = scipy.spatial.KDTree(np.array(list(df_cities['latlong'])))
#         neighbours = kdtree.query(np.array(list(df_clean['tweet_coord'])), k=1)
#         nearest_city = []
#         for i in range(len(neighbours[1])):
#             nearest_city.append(df_cities['name'][i])
#         df_clean['nearest_city'] = nearest_city
#
#         df_clean.drop(columns=['tweet_coord'], inplace=True)
#
#         with self.output().open('w') as f:
#             # crate the final output
#             df_clean.to_csv(f)
#
#     def output(self):
#         return luigi.LocalTarget(self.output_file)

In [114]:
# Closest city sentiment 
# Nearest city logic
import scipy
import numpy as np
s2 = np.array(list(df_cities['latlong']))
s1 = np.array(list(df_airline['tweet_coord']))

kdtree = KDTree(s1)
neighbours = kdtree.query(s2)



In [115]:
print(len(neighbours[1]))
print(len(df_cities['latlong']))

22162
22162


In [116]:
df_airline.columns

Index(['airline_sentiment', 'tweet_coord', 'nearest_city'], dtype='object')

In [117]:
airline_sentiment = []
for i in neighbours[1]:
    airline_sentiment.append(df_airline['airline_sentiment'][i])
    
len(airline_sentiment)

22162

In [79]:
neighbours

(array([ 9.09087552,  9.08799028,  0.64093232, ..., 53.56013258,
        49.21156548, 49.35380831]),
 array([796, 796, 206, ..., 212, 212, 212]))

In [118]:
df_cities['airline_sentiment'] = airline_sentiment

In [119]:
df_cities.columns

Index(['name', 'latitude', 'longitude', 'latlong', 'airline_sentiment'], dtype='object')

In [120]:
df_cities.drop(columns = ['latitude', 'longitude', 'latlong'], inplace= True)

In [121]:
len(set(df_cities['name']))

22162

In [122]:
print(df_cities.groupby(['airline_sentiment']).size())
df_cities = df_cities.drop(df_cities[df_cities['airline_sentiment'] == 'negative'].sample(frac=.54).index)
df_cities = df_cities.drop(df_cities[df_cities['airline_sentiment'] == 'neutral'].sample(frac=.50).index)
print(df_cities.groupby(['airline_sentiment']).size())
df_cities.reset_index(drop=True, inplace= True)

airline_sentiment
negative    9481
neutral     8534
positive    4147
dtype: int64
airline_sentiment
negative    4361
neutral     4267
positive    4147
dtype: int64


In [123]:
print(len(df_cities['name']))
print(len(df_cities['airline_sentiment']))


12775
12775


In [124]:
import pandas as pd
df_features = pd.DataFrame(list(zip(df_cities['name'], df_cities['airline_sentiment'])),columns=['city_name', 'airline_sentiment'])

In [89]:
# To view the full screen
pd.options.display.max_rows = 10000 

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y = le.fit_transform(df_cities['airline_sentiment'])

# One hot encoding 

X = pd.get_dummies(df_cities["name"],prefix='name',drop_first=True)

# df = pd.concat([df_cities,pd.get_dummies(df_cities['name'], prefix='name',drop_first=True)],axis=1)


In [90]:
df = pd.concat([df_cities,pd.get_dummies(df_cities['name'], prefix='name',drop_first=True)],axis=1)



In [ ]:
df_cities_01 = df_cities
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y = le.fit_transform(df_cities['airline_sentiment'])
X = pd.get_dummies(df_cities["name"],prefix='name',drop_first=True)


In [125]:
df_feature = df_cities

df_feature['airline_sentiment'] = le.fit_transform(df_cities['airline_sentiment'])
df_feature = pd.concat([df_cities,pd.get_dummies(df_cities['name'], prefix='name',drop_first=True)],axis=1)
df_feature.drop(columns=['name'], inplace= True)


In [126]:
df_feature.columns

Index(['airline_sentiment', 'name_'s-Gravenzande', 'name_'s-Hertogenbosch',
       'name_Aabenraa', 'name_Aachen', 'name_Aalborg', 'name_Aalen',
       'name_Aalsmeer', 'name_Aalten', 'name_Aarschot',
       ...
       'name_‘Arad', 'name_‘Irbīn', 'name_‘Izbat al Burj', 'name_’Aïn Abid',
       'name_’Aïn Deheb', 'name_’Aïn Merane', 'name_’Aïn el Bell',
       'name_’Aïn el Berd', 'name_’Aïn el Hammam', 'name_’Aïn el Melh'],
      dtype='object', length=12775)

In [127]:
y = df_feature.pop('airline_sentiment')
X = df_feature

In [128]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)

LogisticRegression()

In [129]:
import pickle
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


In [130]:
# Reverse the pandas dummy
name = X.idxmax(axis=1)
city_name = list(map( lambda x: x.replace( 'name_', ''), name))

In [131]:
negative_probability = loaded_model.predict_proba(X)[:,0]
neutral_probability =loaded_model.predict_proba(X)[:,1]
positive_probability =loaded_model.predict_proba(X)[:,2]

In [132]:
df = pd.DataFrame(list(zip(city_name, negative_probability, neutral_probability, positive_probability)), 
                  columns =['city_name', 'negative_probability', 'neutral_probability', 'positive_probability']) 

In [133]:
df.sort_values('positive_probability', inplace=True, ascending=False)

In [134]:
df.head(1000)

,city_name,negative_probability,neutral_probability,positive_probability
6387,Pinerolo,0.250892,0.246308,0.5028
10929,Lubny,0.250892,0.246308,0.5028
10921,Marhanets’,0.250892,0.246308,0.5028
10922,Malyn,0.250892,0.246308,0.5028
10923,Makiyivka,0.250892,0.246308,0.5028
10924,Lyubotyn,0.250892,0.246308,0.5028
10925,Lviv,0.250892,0.246308,0.5028
10926,Lutuhyne,0.250892,0.246308,0.5028
10927,Luts’k,0.250892,0.246308,0.5028
10928,Luhansk,0.250892,0.246308,0.5028


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=1)
clf.fit(X, y)

In [ ]:
df_cities['negative_probability'] = model.predict_proba(X)[:,0]
df_cities['neutral_probability'] = model.predict_proba(X)[:,1]
df_cities['positive_probability'] = model.predict_proba(X)[:,2]

In [ ]:
df_cities.drop(columns =['airline_sentiment'], inplace= True)

In [ ]:
df_cities.sort_values('positive_probability', inplace=True, ascending=False)

In [ ]:
df_cities.to_csv('score.csv' , index=False)

In [ ]:
print(len(model.predict_proba(X)))

In [ ]:
len(y)

In [ ]:
list(le.inverse_transform(model.predict(X)))[0:10]

In [ ]:
0 --> Negetive
1 --> Neutral
2 --> Positive
